In [ ]:
class optimization_problem:
    def __init__(self, initial_state):
        self.initial_state = initial_state
        self.transition_model = self._transition_model()

    def priorities(self):
        s = self.initial_state  # shorthand

        water_priority = 0.33
        fertilizer_priority = 0.33
        irrigation_frequency_priority = 0.33

        # Growth stage
        if s['growth_stage'] == 1:
            water_priority += 0.1
            fertilizer_priority += 0.05
            irrigation_frequency_priority += 0.1
        elif s['growth_stage'] == 2:
            fertilizer_priority += 0.15
            water_priority += 0.05
        elif s['growth_stage'] == 3:
            water_priority += 0.15
            fertilizer_priority += 0.1
            irrigation_frequency_priority += 0.05
        else:
            water_priority -= 0.05
            fertilizer_priority -= 0.1

        # Environmental
        heat_stress = max(0, min(1, (s['temperature'] - 25) / 15))
        water_priority += heat_stress * 0.2
        irrigation_frequency_priority += heat_stress * 0.15

        water_priority -= (s['humidity'] / 100) * 0.1

        rainfall_factor = min(1, s['rainfall_forecast'] / 25)
        water_priority -= rainfall_factor * 0.2
        irrigation_frequency_priority -= rainfall_factor * 0.3

        drought_factor = max(0, 1 - (s['soil_moisture'] / 0.7))
        water_priority += drought_factor * 0.25
        irrigation_frequency_priority += drought_factor * 0.2

        # Soil type
        soil_type = str(s['soil_type'])
        if soil_type == "1":
            irrigation_frequency_priority += 0.15
            water_priority += 0.1
        elif soil_type == "3":
            irrigation_frequency_priority -= 0.1
            water_priority -= 0.05
            fertilizer_priority += 0.05

        # Resource constraints
        water_priority -= s['water_availability'] * 0.1
        if s['water_availability'] < 0.3:
            water_priority += 0.3
            irrigation_frequency_priority += 0.1

        # Irrigation system
        if s['irrigation_system'] == "drip":
            irrigation_frequency_priority += 0.1
            water_priority -= 0.15
        elif s['irrigation_system'] == "flood":
            irrigation_frequency_priority -= 0.2
            water_priority += 0.15

        return {
            'water_priority': max(0.1, water_priority),
            'fertilizer_priority': max(0.1, fertilizer_priority),
            'irrigation_frequency_priority': max(0.1, irrigation_frequency_priority)
        }

    def penalty(self, var, value):
        min_val, max_val = self.optimal_ranges[var]
        if value < min_val:
            return (min_val - value) ** 2
        elif value > max_val:
            return (value - max_val) ** 2
        return 0

    def heuristic(self, state):
        deviation_score = (
            self.penalty('soil_moisture', state['soil_moisture']) +
            self.penalty('N', state['N']) +
            self.penalty('P', state['P']) +
            self.penalty('K', state['K'])
        )

        p = self.priorities()
        cost_penalty = (
            p['water_priority'] * state.get('water_used', 0) +
            p['fertilizer_priority'] * state.get('fertilizer_used', 0)
        )

        if 'WUE' in state and 'WUE' in self.optimal_ranges:
            deviation_score += self.penalty('WUE', state['WUE'])

        return deviation_score + cost_penalty

    def get_actions(self):
        apply_water = [0, 5, 10, 15, 25]
        apply_fertilizer = [0, 5, 10, 15, 20, 25]
        return [(w, f) for w in apply_water for f in apply_fertilizer]

    def apply_action(self, node, action):
        water_added, fertilizer_added = action
        soil_type = str(node['soil_type'])

        new_node = node.copy()

        delta_moisture = 0
        if water_added > 0:
            moisture_per_L = self.transition_model["add_water"]["soil_moisture_increase_per_L"][soil_type]
            delta_moisture = water_added * moisture_per_L
            new_node['soil_moisture'] += delta_moisture

            uptake_per_1pct = self.transition_model["add_water"]["npk_uptake_increase_per_1_percent_moisture"][soil_type]
            for nutrient in ['N', 'P', 'K']:
                new_node[nutrient] += delta_moisture * uptake_per_1pct[nutrient]
                new_node[nutrient] = min(new_node[nutrient], 1.0)

        new_node['water_used'] += water_added

        if fertilizer_added > 0:
            npk_gain = self.transition_model["apply_fertilizer"]["npk_availability_increase"][soil_type]
            for nutrient in ['N', 'P', 'K']:
                new_node[nutrient] += fertilizer_added * npk_gain[nutrient]
                new_node[nutrient] = min(new_node[nutrient], 1.0)

        new_node['fertilizer_used'] += fertilizer_added

        return new_node

    def expand_node(self, node):
        children = []
        for action in self.get_actions():
            child_state = self.apply_action(node, action)
            g_cost = node.get('g', 0) + action[0] * self.initial_state['water_cost'] + action[1] * self.initial_state['fertilizer_cost']
            h = self.heuristic(child_state)
            child_state['g'] = g_cost
            child_state['f'] = g_cost + h
            children.append(child_state)
        return children
    
    def cost_function(self,action,water_availability,fertilizer_availability) :
        if (water_availability == "high" and fertilizer_availability == "high") or (water_availability == "medium" and fertilizer_availability == "medium") or ((water_availability == "low" and fertilizer_availability == "low"))   :
            """
            if both have same availability levels -> we will take into consideration only that water is less expensive then fertilizer 
            """
            water_cost = action["water_added"]
            fertilizer_cost = (action["N_added"] + action["P_added"] + action["K_added"])*2
        elif water_availability == "medium" and fertilizer_availability == "high" or (water_availability == "low" and fertilizer_availability == "medium") :
            """
            Here since water is available in medium levels, we will consider that it costs the same as using fertilizer
            """
            water_cost = action["water_added"]
            fertilizer_cost = action["N_added"] + action["P_added"] + action["K_added"]
        elif (water_availability == "high" and fertilizer_availability == "medium") or (water_availability == "medium" and fertilizer_availability == "low")  :
            water_cost = action["water_added"]
            fertilizer_cost = (action["N_added"] + action["P_added"] + action["K_added"])*3
        elif (water_availability == "low" and fertilizer_availability == "high") :
            water_cost = action["water_added"]*2
            fertilizer_cost = (action["N_added"] + action["P_added"] + action["K_added"])
        elif water_availability == "high" and fertilizer_availability == "low" :
            water_cost = action["water_added"]
            fertilizer_cost = (action["N_added"] + action["P_added"] + action["K_added"])*4

        return water_cost + fertilizer_cost

    def _transition_model(self):
        return {
            "add_water": {
                "units": "1 L/m²",
                "soil_moisture_increase_per_L": {
                    "1": 1.0,
                    "2": 0.6,
                    "3": 0.3
                },
                "npk_uptake_increase_per_1_percent_moisture": {
                    "1": {"N": 0.02, "P": 0.015, "K": 0.018},
                    "2": {"N": 0.025, "P": 0.02, "K": 0.022},
                    "3": {"N": 0.015, "P": 0.025, "K": 0.02}
                }
            },
            "apply_fertilizer": {
                "units": "1 kg/m²",
                "npk_availability_increase": {
                    "1": {"N": 0.15, "P": 0.10, "K": 0.12},
                    "2": {"N": 0.20, "P": 0.18, "K": 0.20},
                    "3": {"N": 0.12, "P": 0.25, "K": 0.18}
                }
            }
        }


state is a dictionnary defined as follows:
{soil_moisture:x,soil_type:"string",water_allocated:x,fertilizer_allocated:x,irrigation_frequency:x,...,all the columns of the dataset}the 

user needs to input the water_cost and the fertilizer_cost, the water_availability,and the fertilizer availability

In [ ]:
class node:
    def __init__(self,state,parent=None,g=0,f=0,):
        self.state = state
        self.parent = parent 
        self.f=f
        self.g=g
    